# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

#### N-grams

In [1]:
import re
import math
from collections import Counter
from itertools import islice

class NGramModel:
    def __init__(self, n=2):
        self.n = n
        self.ngram_counts = Counter()
        self.context_counts = Counter()
        self.vocab = set()
        self.total_words = 0

    def words(self, text):
        """ Breaks the text into words and makes them lowercase """
        return re.findall(r'\w+', text.lower())

    def generate_ngrams(self, words_list):
        """ Generating N-grams from a list of words """
        return zip(*[islice(words_list, i, None) for i in range(self.n)])

    def train(self, corpus):
        """ Model training """
        for sentence in corpus:
            tokens = self.words(" ".join(sentence))
            self.vocab.update(tokens)
            self.total_words += len(tokens)

            for ngram in self.generate_ngrams(tokens):
                self.ngram_counts[ngram] += 1
                self.context_counts[ngram[:-1]] += 1

    def get_prob(self, context, word):
        """ Returns probability P(word | context) """
        context = tuple(context)
        ngram = context + (word,)

        count_ngram = self.ngram_counts.get(ngram, 0)
        count_context = self.context_counts.get(context, 0)

        if count_context == 0:
            return 1e-6 
        return count_ngram / count_context

    def get_log_prob(self, context, word):
        """ Logarithmic probability P(word | context) """
        prob = self.get_prob(context, word)
        return math.log(prob) if prob > 0 else -float('inf')

    def predict_next_word(self, context):
        """ Predicts the next word based on probabilities """
        context = tuple(context)
        candidates = {word: self.get_prob(context, word) for word in self.vocab}
        return max(candidates, key=candidates.get) if candidates else None


def words(text):
    """ Breaks the text into words and makes them lowercase """
    return re.findall(r'\w+', text.lower())

def load_corpus(filename):
    """ Text file downloading """
    with open(filename, 'r', encoding='utf-8') as f:
        return words(f.read())

def build_ngram_model(corpus_file, n=2):
    """ Creating N-gram model """
    model = NGramModel(n)
    with open(corpus_file, 'r', encoding='utf-8') as f:
        sentences = [line.strip().split() for line in f.readlines()]
    model.train(sentences)
    return model

bigram_model = build_ngram_model('/Users/renat/Documents/study/nlp/nlp-assignment-1/dataset/big.txt', 2)
trigram_model = build_ngram_model('/Users/renat/Documents/study/nlp/nlp-assignment-1/dataset/big.txt', 3)

#### Words correction

In [2]:
def known(words):
    """ Filters words that exist in the corpus """
    return set(w for w in words if w in bigram_model.vocab)

def edit_distance(s1, s2):
    dp = [[0 for j in range(len(s2)+1)] for i in range(len(s1)+1)]
    for i in range(1, len(s2)+1):
        dp[0][i] = i
    
    for i in range(1, len(s1)+1):
        dp[i][0] = i

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            dp[i][j] = min(min(dp[i-1][j], dp[i][j-1]) + 1, dp[i-1][j-1] + (1 if s1[i-1] != s2[j-1] else 0))
    
    return dp[len(s1)][len(s2)]

def edits1(word):
    """ All words that are one edit away from the current word """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits  = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts  = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    """ All words that are two edits away from the current word """
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def edits3(word):
    """ All words that are three edits away from the current word """
    return set((e2 for e1 in edits2(word) for e2 in edits2(e1)))

def best_correction(word, prev_word=None, next_word=None):
    """ Selects the best correction using both edit distance and N-gram probability """
    possible_corrections = known([word]) or known(edits1(word)) or known(edits2(word)) or [word]

    if not possible_corrections:
        return word

    def score(w):
        edit_dist = edit_distance(word, w)
        ngram_prob = (
            trigram_model.get_log_prob((prev_word,), w) if prev_word else
            bigram_model.get_log_prob(("",), w)
        )
        return ngram_prob - 2 * edit_dist

    best_word = max(possible_corrections, key=score)

    return best_word


def correct_text(sentence):
    """ Corrects spelling errors in the text considering the context """
    words_list = words(sentence)
    corrected_words = []
    
    for i, word in enumerate(words_list):
        prev_word = words_list[i - 1] if i > 0 else None
        next_word = words_list[i + 1] if i < len(words_list) - 1 else None
        corrected_words.append(best_correction(word, prev_word, next_word))
    
    return " ".join(corrected_words)

test_sentence = "dking sport is very important"
corrected = correct_text(test_sentence)
print("Corrected text:", corrected)

Corrected text: king sport is very important


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

#### Choice of N-gram Dataset
I use big.txt as the dataset because:

- It is a large, real-world corpus containing words from literature, making it suitable for estimating word probabilities.
- It provides a diverse vocabulary, reducing the risk of unseen words.

But it still was not enough, for example "dking species" is "doing species", because this phrase is not known. Alternatives like Wikipedia dumps or modern news articles could be used.

Also, I have had a problem while downloading files from https://www.ngrams.info/download_coca.asp, the site didn't give me the access.

#### Choice of N-gram Model

I implemented both bigram and trigram models:

- Bigram model (n=2) captures basic word dependencies (e.g., "good morning" is more likely than "good turtle").
- Trigram model (n=3) improves contextual understanding by considering two prior words, useful for disambiguation (e.g., "New York city" vs. "New York times").

Using only unigrams would be too simplistic, while higher N-grams (4,5+) require significantly more data and memory.

#### Handling Unknown Words & Probabilities

I address out-of-vocabulary (OOV) words and missing N-grams with:

- Smoothing (1e-6 probability for unseen words): Ensures no word gets a probability of zero, preventing computational errors.
- Backoff Strategy: If a trigram is missing, we rely on a bigram; if the bigram is missing, we fall back to unigrams

#### Edit Distance & Word Correction Strategy

To correct spelling mistakes, I prioritize words based on:
- Existence in the vocabulary (if known, it’s likely correct).
- Single-edit distance (edit1()) words are preferred, as they are more likely to be the intended word.
- Double-edit distance (edit2()) is used only if no single-edit candidates exist.
- Triple-edit distance edit3()
- Trigram/Bigram Probability: Among candidates, I select the word that maximizes P(word | context).

In future, i could also implement dynamic weight assignment for edits, but I assume a uniform penalty for now.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [ ]:
import random
import copy
import re

def introduce_errors(sentence, noise_prob=0.2):
    """ Introduces spelling errors into a sentence with given probability """
    words_list = sentence.split()
    noisy_sentence = []
    
    for word in words_list:
        if random.random() < noise_prob:
            typo_candidates = list(edits1(word))
            if typo_candidates:
                word = random.choice(typo_candidates)
        noisy_sentence.append(word)
    
    return " ".join(noisy_sentence)

def generate_test_set(clean_sentences, noise_prob=0.2):
    """ Generates a test set with noise added """
    return [introduce_errors(sentence, noise_prob) for sentence in clean_sentences]

def evaluate_corrector(corrector, test_sentences, ground_truth):
    """ Evaluates the spelling corrector's accuracy at the word level """
    correct_word_count = 0
    total_word_count = 0

    for i in range(len(test_sentences)):
        corrected_sentence = corrector(test_sentences[i]).split()
        ground_truth_sentence = ground_truth[i].split()

        min_length = min(len(corrected_sentence), len(ground_truth_sentence))

        for j in range(min_length):
            if corrected_sentence[j] == ground_truth_sentence[j]:
                correct_word_count += 1
            total_word_count += 1

    accuracy = correct_word_count / total_word_count if total_word_count > 0 else 0
    return accuracy


def extract_sentences(filename, min_length=5, num_sentences=10):
    """ Extracts random sentences from the given text file. """
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()

    sentences = [s.strip() for s in re.split(r'[.!?]', text) if len(s.strip().split()) >= min_length]
    
    return random.sample(sentences, num_sentences)


holmes_sentences = [
    re.sub(r'[^a-zA-Z0-9\s]', '', s.lower()) 
    for s in extract_sentences('/Users/renat/Documents/study/nlp/nlp-assignment-1/dataset/big.txt', num_sentences=10)
]

test_set_20 = generate_test_set(holmes_sentences, noise_prob=0.2)
test_set_40 = generate_test_set(holmes_sentences, noise_prob=0.4)

accuracy_ngram_20 = evaluate_corrector(correct_text, test_set_20, holmes_sentences)
accuracy_ngram_40 = evaluate_corrector(correct_text, test_set_40, holmes_sentences)

print(f"N-gram Corrector Accuracy (20% noise): {accuracy_ngram_20:.2f}")
print(f"N-gram Corrector Accuracy (40% noise): {accuracy_ngram_40:.2f}")


N-gram Corrector Accuracy (20% noise): 0.95
N-gram Corrector Accuracy (40% noise): 0.80


#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)